#Paquetes necesarios

In [1]:
import cv2  
import math 

from ultralytics import YOLO



AttributeError: 'OutStream' object has no attribute 'reconfigure'

Desde cámara, detección con yolov8 y modelo nano. Visualización propia

In [6]:
# Carga del modelo
model = YOLO('yolov8cars.pt') #Contenedores

# Nombre de las distintas clases
classNames = ['Ambulance', 'Bus', 'Car', 'Motorcycle', 'Truck']


# Captura desde lawebcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Perform inference on an image
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Class name -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

NameError: name 'YOLO' is not defined

Diversos modelos preentrenados, visualizando

In [7]:
# Carga del modelo
#model = YOLO('yolov8n.pt') #Contenedores
#model = YOLO('yolov8n-seg.pt') #Máscaras
model = YOLO('yolov8n-pose.pt')  #Pose

#Para un vídeo 
filename = "C:/Users/otsed/Desktop/RUNNERS_ILUSOS/Multimedia/Bibs/TGC23_PdH_C0056_resultado.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()



NameError: name 'YOLO' is not defined

Intregración con seguimiento (tracking)
Nota: he tenido que bajar a la versión de python 3.9.5 e instalar lap con pip install lap

In [ ]:
# Carga del modelo
model = YOLO('yolov8n.pt') #Contenedores
#model = YOLO('yolov8n-seg.pt') #Máscaras
#model = YOLO('yolov8n-pose.pt')  #Pose

#Para un vídeo 
#filename = "C:/Users/otsed/Desktop/RUNNERS_ILUSOS/Multimedia/Bibs/TGC23_PdH_C0056_resultado.mp4"
filename = "D:/GH010196s.mp4"
results = model.track(source=filename, show=True)  # BoT-SORT tracker (por defecto)
#results = model.track(source=filename, show=True, tracker="bytetrack.yaml")  # ByteTrack tracker

cv2.destroyAllWindows()

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [ ]:
# Tesseract
import cv2
import pytesseract

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('toy.tif') 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Aplica reconocedor a imagen cargada
print(pytesseract.image_to_string(img_rgb))

Reconocimiento decaracteres tras instalar easyocr

In [ ]:
import easyocr

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
result = reader.readtext('toy.tif')
print(result)

#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')